# Bert fine tuning

This notebook was runned before we started to write the paper. Then, the directory names might not correspond to their current names.

In [1]:
# Check that we have a GPU
!nvidia-smi

Mon Jan 25 21:24:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%run -i 'random_state.py'
from packages import *
from clean_functions import *

torch.cuda.is_available()

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


False

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda:0')

Loading NeuralMind's model for Portuguese (https://github.com/neuralmind-ai/portuguese-bert):

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('model_bert_neuralmind/vocab.txt', do_lower_case=False)
bert_model = BertForMaskedLM.from_pretrained('model_bert_neuralmind/').to(device)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [5]:
bert_model.num_parameters()
# => 108 million parameters

109545058

In [8]:
%%time

with open("data/mov_treino.txt", "rb") as fp:   # Unpickling
    mov = pickle.load(fp)
    
max_len=512

texts_mov=[]

total=0

for i in range(len(mov)): #
    for j in range(len(mov[i])):
        
        t=clean_bert(mov[i][j][1])
        total+=1
        texts_mov.append(t) 

texts_mov=pd.Series(texts_mov)

changed_text=pd.Series(texts_mov).apply(lambda x:x+"\n"+"\n")

open('data/train_corpus.txt', "w").write(''.join(changed_text))

CPU times: user 7.63 s, sys: 964 ms, total: 8.59 s
Wall time: 8.7 s


In [22]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=bert_tokenizer,
    file_path="data/train_corpus.txt",
    block_size=512,
)

CPU times: user 28min 11s, sys: 3.68 s, total: 28min 15s
Wall time: 28min 14s


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [24]:
training_args = TrainingArguments(
    output_dir="./model_bert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=4,
    save_steps=-1,
    save_total_limit=2,
)

trainer = Trainer(
    model=bert_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

### Start training

In [25]:
%%time
trainer.train()

{"learning_rate": 4.996556753652252e-05, "loss": 1.946629903705325, "step": 500}
{"learning_rate": 4.9931135073045034e-05, "loss": 1.5757096708603204, "step": 1000}
{"learning_rate": 4.989670260956755e-05, "loss": 1.4144320277136284, "step": 1500}
{"learning_rate": 4.986227014609006e-05, "loss": 1.387831468205899, "step": 2000}
{"learning_rate": 4.982783768261257e-05, "loss": 1.357393349929669, "step": 2500}
{"learning_rate": 4.9793405219135084e-05, "loss": 1.3663083375898823, "step": 3000}
{"learning_rate": 4.97589727556576e-05, "loss": 1.286343130175257, "step": 3500}
{"learning_rate": 4.9724540292180116e-05, "loss": 1.2652709064630325, "step": 4000}
{"learning_rate": 4.969010782870263e-05, "loss": 1.2699733645915985, "step": 4500}
{"learning_rate": 4.965567536522514e-05, "loss": 1.2236726535070557, "step": 5000}
{"learning_rate": 4.962124290174766e-05, "loss": 1.150329444538027, "step": 5500}
{"learning_rate": 4.958681043827017e-05, "loss": 1.2318150872066036, "step": 6000}
{"learni

{"learning_rate": 4.659118611572889e-05, "loss": 0.8747427927890531, "step": 49500}
{"learning_rate": 4.6556753652251405e-05, "loss": 0.9232895197127946, "step": 50000}
{"learning_rate": 4.6522321188773914e-05, "loss": 0.9202062168887205, "step": 50500}
{"learning_rate": 4.648788872529643e-05, "loss": 0.9384902152312862, "step": 51000}
{"learning_rate": 4.6453456261818946e-05, "loss": 0.977715319404495, "step": 51500}
{"learning_rate": 4.641902379834146e-05, "loss": 0.983382265705528, "step": 52000}
{"learning_rate": 4.638459133486398e-05, "loss": 0.9283531812552974, "step": 52500}
{"learning_rate": 4.635015887138649e-05, "loss": 0.9540203712241637, "step": 53000}
{"learning_rate": 4.6315726407908996e-05, "loss": 0.8939281047678669, "step": 53500}
{"learning_rate": 4.628129394443151e-05, "loss": 0.9321549138821283, "step": 54000}
{"learning_rate": 4.624686148095403e-05, "loss": 0.9600951310935779, "step": 54500}
{"learning_rate": 4.6212429017476544e-05, "loss": 0.9522564356806397, "ste

{"learning_rate": 4.3216804694935266e-05, "loss": 0.8925181904716591, "step": 98500}
{"learning_rate": 4.3182372231457775e-05, "loss": 0.8437740849417751, "step": 99000}
{"learning_rate": 4.314793976798029e-05, "loss": 0.8685495511001209, "step": 99500}
{"learning_rate": 4.31135073045028e-05, "loss": 0.8342654318368877, "step": 100000}
{"learning_rate": 4.3079074841025316e-05, "loss": 0.8312324295783182, "step": 100500}
{"learning_rate": 4.304464237754783e-05, "loss": 0.8809644337498758, "step": 101000}
{"learning_rate": 4.301020991407035e-05, "loss": 0.82307657155997, "step": 101500}
{"learning_rate": 4.297577745059286e-05, "loss": 0.8997480523998674, "step": 102000}
{"learning_rate": 4.294134498711537e-05, "loss": 0.8352785526247754, "step": 102500}
{"learning_rate": 4.290691252363789e-05, "loss": 0.8977844277796394, "step": 103000}
{"learning_rate": 4.2872480060160405e-05, "loss": 0.8420131958097918, "step": 103500}
{"learning_rate": 4.2838047596682914e-05, "loss": 0.881771698231197

{"learning_rate": 3.987685573761912e-05, "loss": 0.8534135042590788, "step": 147000}
{"learning_rate": 3.984242327414163e-05, "loss": 0.8165467953867628, "step": 147500}
{"learning_rate": 3.9807990810664146e-05, "loss": 0.8286881811334752, "step": 148000}
{"learning_rate": 3.977355834718666e-05, "loss": 0.8705444675046601, "step": 148500}
{"learning_rate": 3.973912588370918e-05, "loss": 0.8283433238841826, "step": 149000}
{"learning_rate": 3.9704693420231694e-05, "loss": 0.8403270889614941, "step": 149500}
{"learning_rate": 3.967026095675421e-05, "loss": 0.8253813411883312, "step": 150000}
{"learning_rate": 3.963582849327672e-05, "loss": 0.7713796430857037, "step": 150500}
{"learning_rate": 3.9601396029799235e-05, "loss": 0.866889705835376, "step": 151000}
{"learning_rate": 3.9566963566321744e-05, "loss": 0.7949972550484236, "step": 151500}
{"learning_rate": 3.953253110284426e-05, "loss": 0.8102379525220021, "step": 152000}
{"learning_rate": 3.9498098639366776e-05, "loss": 0.8578293781

{"learning_rate": 3.653690678030298e-05, "loss": 0.7749344609154505, "step": 195500}
{"learning_rate": 3.650247431682549e-05, "loss": 0.7832830216216972, "step": 196000}
{"learning_rate": 3.646804185334801e-05, "loss": 0.7947887608012534, "step": 196500}
{"learning_rate": 3.6433609389870523e-05, "loss": 0.7720861472455436, "step": 197000}
{"learning_rate": 3.639917692639304e-05, "loss": 0.7765870911176899, "step": 197500}
{"learning_rate": 3.636474446291555e-05, "loss": 0.7913119195720647, "step": 198000}
{"learning_rate": 3.6330311999438064e-05, "loss": 0.8074491033289233, "step": 198500}
{"learning_rate": 3.6295879535960574e-05, "loss": 0.7533335728097591, "step": 199000}
{"learning_rate": 3.626144707248309e-05, "loss": 0.8431416549878777, "step": 199500}
{"learning_rate": 3.6227014609005605e-05, "loss": 0.7741141673230449, "step": 200000}
{"learning_rate": 3.619258214552812e-05, "loss": 0.7737307393885566, "step": 200500}
{"learning_rate": 3.615814968205064e-05, "loss": 0.8319742536

...

{"learning_rate": 1.0023909902638767e-05, "loss": 0.6151483952075941, "step": 580500}
{"learning_rate": 9.98947743916128e-06, "loss": 0.5814437149640871, "step": 581000}
{"learning_rate": 9.955044975683795e-06, "loss": 0.6434655097271316, "step": 581500}
{"learning_rate": 9.92061251220631e-06, "loss": 0.6274386137584225, "step": 582000}
{"learning_rate": 9.886180048728824e-06, "loss": 0.524150544114178, "step": 582500}
{"learning_rate": 9.851747585251336e-06, "loss": 0.6297437805389055, "step": 583000}
{"learning_rate": 9.817315121773852e-06, "loss": 0.5545966559965163, "step": 583500}
{"learning_rate": 9.782882658296364e-06, "loss": 0.6013038169840583, "step": 584000}
{"learning_rate": 9.748450194818879e-06, "loss": 0.6292839660121826, "step": 584500}
{"learning_rate": 9.714017731341393e-06, "loss": 0.6191668774143327, "step": 585000}
{"learning_rate": 9.679585267863907e-06, "loss": 0.5830070460846182, "step": 585500}
{"learning_rate": 9.64515280438642e-06, "loss": 0.5660908025117823,

{"learning_rate": 6.683960945322626e-06, "loss": 0.5833994100344134, "step": 629000}
{"learning_rate": 6.64952848184514e-06, "loss": 0.5892561943371547, "step": 629500}
{"learning_rate": 6.615096018367653e-06, "loss": 0.573047022413928, "step": 630000}
{"learning_rate": 6.580663554890168e-06, "loss": 0.6249527226218488, "step": 630500}
{"learning_rate": 6.5462310914126816e-06, "loss": 0.5897945310912328, "step": 631000}
{"learning_rate": 6.511798627935195e-06, "loss": 0.5470072530498729, "step": 631500}
{"learning_rate": 6.47736616445771e-06, "loss": 0.6168138919294579, "step": 632000}
{"learning_rate": 6.442933700980223e-06, "loss": 0.5472594911990455, "step": 632500}
{"learning_rate": 6.408501237502737e-06, "loss": 0.5904203704475658, "step": 633000}
{"learning_rate": 6.374068774025252e-06, "loss": 0.5647705920536537, "step": 633500}
{"learning_rate": 6.339636310547765e-06, "loss": 0.6077680653878488, "step": 634000}
{"learning_rate": 6.30520384707028e-06, "loss": 0.5855768364662072,

{"learning_rate": 3.344011988006485e-06, "loss": 0.560349117259495, "step": 677500}
{"learning_rate": 3.3095795245289982e-06, "loss": 0.5806249769571005, "step": 678000}
{"learning_rate": 3.2751470610515124e-06, "loss": 0.5890685299703619, "step": 678500}
{"learning_rate": 3.2407145975740267e-06, "loss": 0.5909782031949143, "step": 679000}
{"learning_rate": 3.206282134096541e-06, "loss": 0.5904643914634362, "step": 679500}
{"learning_rate": 3.1718496706190543e-06, "loss": 0.5651151937035611, "step": 680000}
{"learning_rate": 3.1374172071415685e-06, "loss": 0.5581625293799444, "step": 680500}
{"learning_rate": 3.1029847436640823e-06, "loss": 0.5478057687387337, "step": 681000}
{"learning_rate": 3.0685522801865965e-06, "loss": 0.5834472351633012, "step": 681500}
{"learning_rate": 3.0341198167091108e-06, "loss": 0.5534010106247151, "step": 682000}
{"learning_rate": 2.9996873532316246e-06, "loss": 0.5607153835395584, "step": 682500}
{"learning_rate": 2.965254889754139e-06, "loss": 0.582829

{"learning_rate": 3.8495494167829335e-08, "loss": 0.5429929321088129, "step": 725500}
{"learning_rate": 4.063030690343347e-09, "loss": 0.5804766270373948, "step": 726000}


CPU times: user 1d 14h 18min 28s, sys: 4min 12s, total: 1d 14h 22min 40s
Wall time: 1d 14h 17min 43s


TrainOutput(global_step=726059, training_loss=0.7227578997399938)

#### Save final model (+ tokenizer + config) to disk

For the paper, the saved model is in directory './models':

In [26]:
trainer.save_model("./model_bert")